In [45]:
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import clear_output

import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from visualkeras import layered_view
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.datasets import mnist, cifar10, fashion_mnist
from tensorflow.keras.layers import Dense, Flatten, UpSampling2D, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.applications import ResNet50, MobileNet, MobileNetV2, MobileNetV3Small

from gand.data import data
from gand.config import MLConfig
from gand.preprocessing import utils
from gand.visualisation import visualise
from gand.models import models, architecture

import pandas as pd
from tabulate import tabulate
from tqdm.keras import TqdmCallback
from sklearn.metrics import classification_report

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
dataset = fashion_mnist
EPOCHS = 100
BATCH_SIZE = MLConfig.BATCH_SIZE

loss = "categorical_crossentropy"
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.0001, momentum=0.9)
metrics = ["accuracy"]

g_model_mnist = load_model(Path.cwd() / 'notebooks/models/cgan/mnist/gen_model_e-200.h5')
g_model_fashion_mnist = load_model(Path.cwd() / 'notebooks/models/cgan/fashion_mnist/gen_model_e-536.h5')
g_model_cifar10 = load_model(Path.cwd() / 'notebooks/models/cgan/cifar10/gen_model_e-553.h5')
g_model = g_model_fashion_mnist

train_with_gan = False
imbalance_data = True

reshape_image = True

train_type = 0

TRANSFER_LEARNING_MODEL = ResNet50
TRANSFER_LEARNING_PREPROCESSOR = tf.keras.applications.resnet50.preprocess_input

In [52]:
name = 'normal'
if train_with_gan:
    name = 'gans'
elif imbalance_data:
    name = 'imbalanced'

MLConfig.TYPE_NAMES = [f'{name}-{TRANSFER_LEARNING_MODEL().name.capitalize()}']
print(MLConfig.TYPE_NAMES)

['imbalanced-Resnet50']


# Training

### GAN Data

In [53]:
X_train_gan, y_train_gan = None, None
if train_with_gan:
    imbalance_data = True
    # GAN DATA
    n = 3000
    X_train_gan, y_train_gan = data.generate_fake_data(n=n, g_model=g_model, seed=10, verbose=1)
    print(X_train_gan.shape, y_train_gan.shape)
    
    X_train_gan = (X_train_gan+1) * 127.5
    
    if X_train_gan.shape[-1] == 1:
        X_train_gan = np.repeat(X_train_gan, 3, axis=-1)

        if reshape_image:
            X_train_gan = np.array(tf.image.resize(X_train_gan, [32,32]))
            
    X_train_gan = TRANSFER_LEARNING_PREPROCESSOR(X_train_gan)
    y_train_gan = to_categorical(y_train_gan, num_classes=10)
    
    print(X_train_gan.min(), X_train_gan.max(), X_train_gan.shape, y_train_gan.shape)

### Normal Data

In [54]:
((X_train_real, y_train_real), (X_test_real, y_test_real)), dataset_name = data.load_dataset(dataset, return_name=True)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

if len(X_train_real.shape) == 3:
    X_train_real = np.expand_dims(X_train_real, axis=-1)
    X_test_real = np.expand_dims(X_test_real, axis=-1)
    
    X_train_real = np.repeat(X_train_real, 3, axis=-1)
    X_test_real = np.repeat(X_test_real, 3, axis=-1)

    if reshape_image:
        X_train_real = np.array(tf.image.resize(X_train_real, [32,32]))
        X_test_real = np.array(tf.image.resize(X_test_real, [32,32]))

X_train_real = TRANSFER_LEARNING_PREPROCESSOR(X_train_real)
X_test_real = TRANSFER_LEARNING_PREPROCESSOR(X_test_real)

print(X_train_real.min(), X_train_real.max(), X_test_real.min(), X_test_real.max())

y_train_real = to_categorical(y_train_real, num_classes=10)
y_test_real = to_categorical(y_test_real, num_classes=10)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

if imbalance_data:
    # REMOVING 50%
    # keep_per_class = 2000
    keep_per_class = (X_train_real.shape[0] // (2 * 10))
    X_train_removed, y_train_removed = [], []
    for i in range(y_train_real.shape[-1]):
        class_idx = np.where(np.argmax(y_train_real, axis=-1) == i)[0]
        selected_idx = np.random.choice(class_idx, keep_per_class)
        X_train_removed.append(X_train_real[selected_idx])
        y_train_removed.append(y_train_real[selected_idx])
    
    X_train_real, y_train_real = np.concatenate(X_train_removed, axis=0), np.concatenate(y_train_removed, axis=0)

print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,) fashion_mnist
-123.68 151.061 -123.68 151.061
(60000, 32, 32, 3) (60000, 10) (10000, 32, 32, 3) (10000, 10) fashion_mnist
(30000, 32, 32, 3) (30000, 10) (10000, 32, 32, 3) (10000, 10) fashion_mnist


### Appending GAN Data

In [55]:
if train_with_gan:
    X_train_real = np.concatenate((X_train_real, X_train_gan), axis=0)
    y_train_real = np.concatenate((y_train_real, y_train_gan), axis=0)
    print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

### Shuffle It

In [56]:
X_train_real, y_train_real = shuffle(X_train_real, y_train_real)
X_test_real, y_test_real = shuffle(X_test_real, y_test_real)
print(X_train_real.shape, y_train_real.shape, X_test_real.shape, y_test_real.shape, dataset_name)

(30000, 32, 32, 3) (30000, 10) (10000, 32, 32, 3) (10000, 10) fashion_mnist


### Make it Faster

In [57]:
# train_data = tf.data.Dataset.from_tensor_slices((X_train_real, y_train_real))
# test_data = tf.data.Dataset.from_tensor_slices((X_test_real, y_test_real))

# train_data = train_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# train_data, test_data, dataset_name

In [58]:
MLConfig.TYPE_NAMES[train_type]

'imbalanced-Resnet50'

In [59]:
X_train_real.shape[1:]

(32, 32, 3)

In [36]:
base_model = TRANSFER_LEARNING_MODEL(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable=False
# upsamp = 7
# if X_train_real.shape[1] == 28:
#     upsamp = 8

input = Input(shape=X_train_real.shape[1:])
# layer = UpSampling2D(size=(upsamp, upsamp))(input)
assert input.shape == (None, 32, 32, 3)

layer = base_model(input)
layer = GlobalAveragePooling2D( )(layer)
layer = Flatten()(layer)
# layer = Dense(512, activation='relu')(layer)
# layer = Dropout(0.25)(layer)
layer = Dense(512)(layer)
layer = Dropout(0.25)(layer)
output = Dense(10, activation='softmax')(layer)

model = Model(input, output, name=MLConfig.TYPE_NAMES[train_type])

In [61]:
base_model = TRANSFER_LEARNING_MODEL(weights='imagenet', include_top=False, 
                                     input_shape=(32, 32, 3))
base_model.trainable=False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Flatten(),
    # Dense(1024, activation='relu'),
    # Dropout(0.25),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
], name=MLConfig.TYPE_NAMES[train_type])

In [62]:
opt = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss=loss, metrics=metrics)
model.summary()

Model: "imbalanced-Resnet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_6 (Flatten)         (None, 2048)              0         
                                                                 
 dense_12 (Dense)            (None, 512)               1049088   
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                               

In [63]:
history = model.fit(X_train_real, y_train_real, batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, validation_data=(X_test_real, y_test_real), verbose=0, 
                    callbacks=[TqdmCallback(verbose=1)], shuffle=True)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

KeyboardInterrupt: 

In [ ]:
eval_data = ((X_train_real, y_train_real), (X_test_real, y_test_real))
models.save_metrics(dataset_name=dataset_name, train_type=train_type,
                   epochs=EPOCHS, model=model, eval_data=eval_data, 
                   history=history)

In [ ]:
history_path = Path.cwd() / f'reports/history/{dataset_name}/{MLConfig.TYPE_NAMES[train_type]}/E_{EPOCHS:03d}/'
history_path.mkdir(parents=True, exist_ok=True)
pd.DataFrame.from_dict(history.history).to_csv(history_path / f'{model.name}.csv',index=False)

In [ ]:
visualise.metric_plot(show_fig=True, history=history, dataset_name=dataset_name, 
                      savefig=False, epochs=EPOCHS)

In [ ]:
# model = load_model(Path.cwd() / Path(f'reports/models/{dataset_name}/{MLConfig.TYPE_NAMES[train_type]}/E_{EPOCHS:03d}/{model.name}.h5'))
# model.name

In [ ]:
model = load_model(Path.cwd() / Path(f'reports/models/Resnet50-savious.h5'))
model.layers

In [ ]:
model.layers[2].activation.__name__, model.layers[-1].activation.__name__

In [ ]:
# _, acc = model.evaluate(X_train_real, y_train_real)
# _, acc = model.evaluate(X_test_real, y_test_real)

# y_pred = np.argmax(model.predict(X_train_real), axis=-1)
# report = classification_report(np.argmax(y_train_real, axis=1), y_pred)
# print(report)

# y_pred = np.argmax(model.predict(X_test_real), axis=-1)
# report = classification_report(np.argmax(y_test_real, axis=1), y_pred)
# print(report)